# Guiding Question: What are customers saying about our movies?

## Our approach to analyzing:
- Analyze text of movie reviews
- Clean the review text
- HOW: Topic Modeliung
- Label the most reviews with the most important topics
- Visualize the results

In [1]:
%pwd

'/Users/ekselan/Documents/GitHub/DS-Unit-4-Machine-Learning/1-NLP/DS-Unit-4-Sprint-1-NLP/module4-topic-modeling'

In [2]:
import pandas as pd

PATH = "/Users/ekselan/Desktop/LAMBDA/134715_320111_compressed_IMDB Dataset.csv.zip"
df = pd.read_csv(PATH)

print(df.shape)
df.head()

(50000, 2)


,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


## Clean Text Data

In [ ]:
# Remove br tags
"""
1. .replace method
2. BeautifulSoup get text
3. Regex
"""

In [3]:
from bs4 import BeautifulSoup

def clean_description(desc):
    soup = BeautifulSoup(desc)
    return soup.get_text()

In [4]:
df['review'] = df['review'].apply(clean_description)

In [5]:
df.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. The filming tec...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


In [14]:
samp = df.sample(15000)

## Tokenization

- Spacy
- Pure Python (stopwords, punct, etc)
- Gensim
- NLTK (not recommended)

In [7]:
import spacy

nlp = spacy.load("en_core_web_lg") #> md and lg use good word embeddings

In [15]:
## Use Lemmas as our tokens

tokens = []

for doc in nlp.pipe(samp['review']):
    
    doc_tokens = []
    
    for token in doc:
        if (token.is_stop == False) & (token.is_punct == False) & (token.pos != 'PRON'):
            doc_tokens.append(token.lemma_.strip())
            
    tokens.append(doc_tokens)

In [19]:
# Sanity check that our function worked properly

len(tokens) == samp.shape[0]

True

## Gensim LDA Topic Modeling

In [16]:
import gensim

from gensim import corpora
from gensim.models.ldamulticore import LdaMulticore

Sklearn's 'CountVectorizer' v. 'Gensim.corpora'

| 'CountVectorizer' | 'Gensim.corpora' | Description|
| --- | --- |
| Instance | - | N/A

In [17]:
id2word = corpora.Dictionary(tokens)

In [18]:
len(id2word.keys())

71879

In [23]:
id2word.filter_extremes(no_below=10, no_above=.95)

In [24]:
len(id2word.keys())

11468

In [25]:
corpora = [id2word.doc2bow(doc) for doc in tokens]

In [ ]:
corpora[0]

In [26]:
lda = LdaMulticore(corpus=corpora,
                   id2word=id2word,
                   num_topics=15,
                   passes=100,
                   workers=12)

In [27]:
lda.print_topics()

[(0,
  '0.014*"play" + 0.014*"game" + 0.013*"movie" + 0.011*"like" + 0.009*"school" + 0.008*"music" + 0.008*"love" + 0.008*"great" + 0.006*"girl" + 0.006*"song"'),
 (1,
  '0.080*"movie" + 0.020*"bad" + 0.018*"like" + 0.016*"good" + 0.015*"watch" + 0.015*"film" + 0.012*"think" + 0.011*"see" + 0.010*"time" + 0.008*"character"'),
 (2,
  '0.051*"film" + 0.016*"horror" + 0.015*"movie" + 0.013*"see" + 0.013*"good" + 0.009*"watch" + 0.008*"great" + 0.008*"year" + 0.007*"time" + 0.007*"dvd"'),
 (3,
  '0.009*"kill" + 0.007*"like" + 0.007*"get" + 0.007*"scene" + 0.007*"film" + 0.006*"bad" + 0.006*"look" + 0.006*"movie" + 0.006*"guy" + 0.005*"man"'),
 (4,
  '0.040*"movie" + 0.024*"funny" + 0.022*"like" + 0.015*"comedy" + 0.013*"laugh" + 0.012*"watch" + 0.012*"good" + 0.010*"bad" + 0.009*"think" + 0.009*"time"'),
 (5,
  '0.014*"film" + 0.006*"scene" + 0.006*"New" + 0.006*"good" + 0.005*"go" + 0.005*"play" + 0.005*"man" + 0.004*"car" + 0.004*"thriller" + 0.004*"plot"'),
 (6,
  '0.037*"film" + 0.006

In [28]:
import re
words = [re.findall('"([^"]*)"',t[1]) for t in lda.print_topics()]
topics = [' '.join(t[0:5]) for t in words]

In [29]:
for id, t in enumerate(topics):
    print(f"----- Topic {id} -----")
    print(t, end="\n\n")

----- Topic 0 -----
play game movie like school

----- Topic 1 -----
movie bad like good watch

----- Topic 2 -----
film horror movie see good

----- Topic 3 -----
kill like get scene film

----- Topic 4 -----
movie funny like comedy laugh

----- Topic 5 -----
film scene New good go

----- Topic 6 -----
film story time work life

----- Topic 7 -----
series episode tv show season

----- Topic 8 -----
film play role performance good

----- Topic 9 -----
film character like good story

----- Topic 10 -----
book play story good character

----- Topic 11 -----
film good western great short

----- Topic 12 -----
movie action good fight bad

----- Topic 13 -----
film life love man people

----- Topic 14 -----
movie love kid like child



## Interpret LDA Results

1. Topic Term Distribution (how good is our LDA model?) / what are the topics
2. Document Topic Distribution
    - What are the documents about?
    - What are the most common themes(topics)?
    - What topics are associated with positive and negative sentiment?

In [30]:
# Part 1: Topic Disctance Visualization
"""
Tells us if the topics are disctinct, and what terms are most import
"""

import pyLDAvis.gensim

pyLDAvis.enable_notebook()

In [32]:
pyLDAvis.gensim.prepare(lda, corpora, id2word)

# Generally don't want overlapping topics

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
1     -0.139537 -0.024118       1        1  16.613131
6      0.071903 -0.062146       2        1  11.967257
9     -0.013267 -0.077504       3        1  10.191090
13     0.028895  0.021735       4        1  10.040487
3     -0.066494 -0.105562       5        1   9.702102
4     -0.151273  0.043978       6        1   6.528385
2     -0.069382 -0.026875       7        1   5.725814
8      0.183138  0.074582       8        1   4.738335
10     0.109194  0.063526       9        1   4.621392
5      0.089140 -0.086562      10        1   4.262901
14    -0.089461  0.170458      11        1   4.038042
0     -0.006399  0.129226      12        1   3.463425
7     -0.002825  0.033460      13        1   3.382325
12    -0.079643 -0.099602      14        1   2.364276
11     0.136012 -0.054594      15        1   2.361042, topic_info=       Term          Freq         Total Category  logprob  loglift
33    movie  30312.000000  30312.000000  Default  30.0000  30.0000
110    film  28112.000000  28112.000000  Default  29.0000  29.0000
6       bad   7775.000000   7775.000000  Default  28.0000  28.0000
761  series   1901.000000   1901.000000  Default  27.0000  27.0000
17    funny   2893.000000   2893.000000  Default  26.0000  26.0000
..      ...           ...           ...      ...      ...      ...
422   shoot     90.377037   1307.624878  Topic15  -5.9118   1.0741
29     like    118.140297  13150.840820  Topic15  -5.6440  -0.9663
659     see    108.339447   7379.325195  Topic15  -5.7306  -0.4751
496    cast     96.108162   2357.471924  Topic15  -5.8504   0.5462
30     look    100.862640   5751.195801  Topic15  -5.8021  -0.2973

[1254 rows x 6 columns], token_table=      Topic      Freq Term
term                      
173       2  0.116903     
173       3  0.025506     
173       4  0.012753     
173       5  0.221054     
173       8  0.607897     
...     ...       ...  ...
343       3  0.049339    
343       4  0.006492    
343       5  0.068814    
343       8  0.009089    
343      14  0.014282    

[4999 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[2, 7, 10, 14, 4, 5, 3, 9, 11, 6, 15, 1, 8, 13, 12])

In [ ]:
# Part 2: What are the documents about?
# Eqv. to a `.predict` statement in sklearn
# Scoring the topic distribution of a single document

lda[corpora[1]]

In [34]:
distro = [lda[d] for d in corpora] #> score all docs in corpus

In [35]:
def update(doc):
    d_dist = {k:0 for k in range (0,15)}
    for t in doc:
        d_dist[t[0]] = t[1]
    return d_dist

new_distro = [update(d) for d in distro]

In [36]:
doc_topics = pd.DataFrame.from_records(new_distro)
doc_topics.columns = topics

In [37]:
doc_topics.head()

,play game movie like school,movie bad like good watch,film horror movie see good,kill like get scene film,movie funny like comedy laugh,film scene New good go,film story time work life,series episode tv show season,film play role performance good,film character like good story,book play story good character,film good western great short,movie action good fight bad,film life love man people,movie love kid like child
0,0.286967,0.000000,0.000000,0.213201,0.293703,0.0,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.0,0.195935,0.000000
1,0.150832,0.000000,0.000000,0.031807,0.000000,0.0,0.040211,0.000000,0.419399,0.00000,0.230947,0.000000,0.0,0.115591,0.000000
2,0.000000,0.000000,0.096118,0.197774,0.000000,0.0,0.363851,0.000000,0.170439,0.00000,0.000000,0.034946,0.0,0.000000,0.133371
3,0.395350,0.000000,0.000000,0.000000,0.248255,0.0,0.000000,0.055433,0.000000,0.00000,0.121306,0.000000,0.0,0.170517,0.000000
4,0.102382,0.387505,0.000000,0.257865,0.000000,0.0,0.000000,0.000000,0.000000,0.04157,0.000000,0.000000,0.0,0.201652,0.000000


In [39]:
doc_topics['primary_topic'] = doc_topics.idxmax(axis=1)

In [40]:
doc_topics['primary_topic'].value_counts() #> getting idea of most important topics

movie bad like good watch          3935
film story time work life          1693
film character like good story     1615
film life love man people          1323
kill like get scene film           1184
movie funny like comedy laugh      1127
film horror movie see good          872
book play story good character      631
movie love kid like child           534
film play role performance good     512
film scene New good go              387
series episode tv show season       379
play game movie like school         366
film good western great short       223
movie action good fight bad         219
Name: primary_topic, dtype: int64

## Selecting the Number of Topics (Learn)

In [ ]:
from gensim.models.coherencemodel import CoherenceModel

def compute_coherence_values(dictionary, corpus, limit, start=2, step=3, passes=5):
    """
    Compuite u_mass coherence for various number of topics
    
    Parameters:
    -----------
    dictionary: Gensim dictionary
    corpus: Gensim corpus
    limit: Max number of topic
    passes: the number of times the entire lda model & coherence values are calculated
    
    Returns:
    ---------
    coherence_values
    """

In [ ]:
# Will take a long time to run - test first with a very small sample

coherence_values = compute_coherence_values(dictionary=id2word,
                                                       corpus=corpora,
                                                       start=3,
                                                       limit=40,
                                                       step=2,
                                                       passes=1)

In [ ]:
topic_coherence = pd.DataFrame.from_records(coherence_values)

In [ ]:
topic_coherence.head()

In [ ]:
### looking for the highest point past zero when looking at this plot

import seaborn as sns

az = sns.lineplot(x="num_topic", y="coherence_score", data=topic_coherence)

### Once you get your optimal number of topic, re-run the lda model with
### that number, crank up num of passes to get very stable results, and
### then interpret

In [ ]:
### tokenization, document classification, and a 
### little lda (gensim api / analyzing results of model)